# Webscraping & Applied ML

## Description : 

La description du projet se trouve dans le word 

## Scrapping

Our sources are Google Flight and ICAO.

Google flights : https://www.google.com/travel/flights?hl=fr

ICAO : https://www.icao.int/environmental-protection/Carbonoffset/Pages/default.aspx

The objective is to scrap some data that we can use for our case.

Imports :

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import pandas as pd
import time
import logging
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time

In [2]:
# Initialize the WebDriver
service = Service(r'C:\Users\deslo\.wdm\drivers\chromedriver\win64\131.0.6778.108\chromedriver-win32\chromedriver.exe')
driver = webdriver.Chrome(service=service)

# Open Google Flights
driver.get('https://www.google.com/travel/flights?hl=en')

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time
import pandas as pd

#part to scrap data from google flights :
def input_search_criteria(origin, destination, travel_departure):

    date_input = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '[aria-label="Departure"]'))
    )
    date_input.clear()
    date_input.send_keys(travel_departure)
    time.sleep(1)
    date_input.send_keys(Keys.ENTER)


    origin_input = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '[aria-label="Where from?"]'))
    )
    origin_input.clear()
    origin_input.send_keys(origin)
    origin_input.send_keys(Keys.ENTER)

    time.sleep(1)
    destination_input = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '[aria-label="Where to?"]'))
    )
    destination_input.clear()
    destination_input.send_keys(destination)
    time.sleep(1)
    #click on the first suggestion
    click_first_suggestion = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '[class="zsRT0d"]'))
    )
    click_first_suggestion.click()
    time.sleep(1)




def scrape_flight_data():
    flight_data = []
    all_flights = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, '[aria-label="View flights"]'))
    )
    all_flights.click()
    time.sleep(1)
    
    all_filters_button = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, 'VfPpkd-LgbsSe VfPpkd-LgbsSe-OWXEXe-Bz112c-M1Soyc VfPpkd-LgbsSe-OWXEXe-dgl2Hf ksBjEc lKxP2d LQeN7 bRx3h x4Vnpe yJQRU sIWnMc hNyRxf cd29Sd'))
    )
    all_filters_button.click()

    less_emissions = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, '[aria-label="Less emissions only"]'))
    )
    less_emissions.click()



    time.sleep(5)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    flights = soup.find_all('li', class_='pIav2d')
    for flight in flights:
            airline = flight.find('div', class_='mxvQLc ceis6c uj4xv uVdL1c A8qKrc sMVRZe').text.strip()
            price = flight.find('div', class_='YMlIz FpEdX jLMuyc').text.strip()
            departure_time = flight.find('div', class_='wtdjmc YMlIz ogfYpf tPgKwe').text.strip()
            arrival_time = flight.find('div', class_='G2WY5c sSHqwe ogfYpf tPgKwe').text.strip()
            duration = flight.find('div', class_='Ak5kof').text.strip()
            emissions = flight.find('div', class_='N6PNV ogfYpf juCwOd BYjnCf').text.strip()

            flight_data.append({
                'Airline': airline,
                'Price': price,
                'Departure Time': departure_time,
                'Arrival Time': arrival_time,
                'Duration': duration,
                'Emissions': emissions
            })
    return flight_data

input_search_criteria('Paris', 'Barcelona', '10/02/2025')

# Scrape flight data
flight_data = scrape_flight_data()

# Convert to DataFrame
df = pd.DataFrame(flight_data)
print(df)

# Close the WebDriver


TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x008034A3+25059]
	(No symbol) [0x0078CEA4]
	(No symbol) [0x0066BEC3]
	(No symbol) [0x006AFD86]
	(No symbol) [0x006AFFCB]
	(No symbol) [0x006ED952]
	(No symbol) [0x006D1F44]
	(No symbol) [0x006EB51E]
	(No symbol) [0x006D1C96]
	(No symbol) [0x006A3FAC]
	(No symbol) [0x006A4F3D]
	GetHandleVerifier [0x00AF5613+3113811]
	GetHandleVerifier [0x00B0A2DA+3199002]
	GetHandleVerifier [0x00B02AB2+3168242]
	GetHandleVerifier [0x008A3310+680016]
	(No symbol) [0x007957ED]
	(No symbol) [0x00792A98]
	(No symbol) [0x00792C35]
	(No symbol) [0x00785890]
	BaseThreadInitThunk [0x76A55D49+25]
	RtlInitializeExceptionChain [0x77AECEBB+107]
	RtlGetAppContainerNamedObjectPath [0x77AECE41+561]


In [4]:
driver.get('https://www.google.com/travel/flights?hl=en')